In [2]:
import MetaTrader5 as mt5
import pandas as pd
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [3]:
mt5.initialize(
            path=os.getenv("MT5_PATH"),
            login=int(os.getenv("MT5_LOGIN")),
            password=os.getenv("MT5_PASSWORD"),
            server=os.getenv("MT5_SERVER"),
            timeout=int(os.getenv("MT5_TIMEOUT")),
            portable=eval(os.getenv("MT5_PORTABLE")),
        )

True

In [9]:
bars = pd.DataFrame(mt5.copy_rates_from_pos(
    "EURUSD",
    mt5.TIMEFRAME_H1,
    1,
    50,
))

bars['time'] = pd.to_datetime(bars['time'], unit='s')
bars.set_index('time', inplace=True)
bars.rename(columns={
    'tick_volume': 'tickvol',
    'real_volume': 'vol'
}, inplace=True)
bars = bars[['open', 'high', 'low', 'close', 'tickvol', 'vol', 'spread']]
bars.head()

,open,high,low,close,tickvol,vol,spread
time,,,,,,,
2025-04-03 11:00:00,1.09708,1.10206,1.09706,1.10093,5266,0,0
2025-04-03 12:00:00,1.10093,1.10474,1.10039,1.10305,5551,0,0
2025-04-03 13:00:00,1.10305,1.11459,1.10279,1.10519,8680,0,0
2025-04-03 14:00:00,1.10526,1.11010,1.10388,1.10875,6422,0,0
2025-04-03 15:00:00,1.10875,1.11096,1.10730,1.10817,7340,0,0


In [10]:
type(bars.iloc[-1])

pandas.core.series.Series

In [13]:
tick = mt5.symbol_info_tick("EURUSD")._asdict()
tick

{'time': 1744034613,
 'bid': 1.0961,
 'ask': 1.09612,
 'last': 0.0,
 'volume': 0,
 'time_msc': 1744034613850,
 'flags': 4,
 'volume_real': 0.0}

In [21]:
def get_latest_tick(symbol: str) -> dict:
    """Get the latest tick for a given symbol.
    Args:
        symbol (str): The symbol to fetch data for.
    Returns:
        dict: The latest tick data.
    Raises:
        ValueError: If the symbol is invalid.
        RuntimeError: If there is an error fetching data from MT5.
    """
    try:
        tick = mt5.symbol_info_tick(symbol)
        if tick is None:
            raise ValueError(f"No tick data received for symbol: {symbol}.")

    except Exception as e:
        raise RuntimeError(
            f"Error fetching tick data for symbol: {symbol}. "
            f"MT5 Error: {mt5.last_error()} "
            f"Exception: {str(e)}"
        )

    else:
        return tick._asdict()  # Convert the tick object to a dictionary

In [22]:
get_latest_tick("EURUSD")

{'time': 1744034892,
 'bid': 1.09645,
 'ask': 1.09645,
 'last': 0.0,
 'volume': 0,
 'time_msc': 1744034892490,
 'flags': 4,
 'volume_real': 0.0}